In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
from models.rnn.combined_networks import train_report, Loss
import wandb
%load_ext autoreload
%autoreload 2

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:100]
# wandb.init()

## Plain RNN

In [3]:
algo = RNN_Plain(symbols = "+*-0123456789t", 
output_sequence_length = 9, 
encoded_seq_length = 9, 
num_epochs = 2, 
input_size = input_lang.n_words, 
hidden_size = 512, 
output_size=output_lang.n_words, 
embedding_size = 256, 
batch_size = 64, 
num_gru_layers = 1,
dropout_prob = 0.,
loss=Loss.MAE,
bidirectional=False,
binary_encoding=True,
wandb_activate = False)


Num_batch:  64
EncoderRNN(
  (embedding): Embedding(4300, 256)
  (gru): GRU(32, 512)
)
DecoderRNN(
  (embedding): Embedding(16, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=16, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [5]:
train_report(algo, input_lang, output_lang, train, X_test, y_test, 300000)


AttributeError: 'int' object has no attribute 'backward'

In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:10]

['t*6+8-6-t',
 '1*t+0*3+3',
 't*3*t+5-t',
 '4*6+t-8+t',
 '4*1+2+7-t',
 '7-7*3-t+8',
 '4*1+2+7-t',
 't-t+4-t+3',
 't*8-t*2+7',
 't+2-4*2*t']

In [10]:
accuracy_score(pred, y_test[:1000])

NameError: name 'pred' is not defined

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

# Helper Stuff

In [4]:
import torch
import torch.nn as nn
m = nn.LogSoftmax()
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

In [42]:
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()

# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)

# print(data.shape)
activated = m(conv(data))
print("output: ", activated.shape)

# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
print("target: ", target.shape)
output = loss(activated, target)

# output.backward()

In [41]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
# input = minibatchsize, number of categories

input = torch.randn(3, 5, requires_grad=True)
# print(input.shape)
activation = m(input)
print(activation.shape)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(target.shape)
output = loss(activation, target)
# output.backward()

In [10]:
from line_profiler import LineProfiler

stuff_to_evaluate = [1,2,3]
def function_to_evaluate(stuff_to_evaluate):
    pass

lp = LineProfiler()
lp_wrapper = lp(function_to_evaluate)
lp_wrapper(stuff_to_evaluate)
lp.print_stats()

Timer unit: 1e-07 s

Total time: 5.2e-06 s
File: <ipython-input-10-550e04f4f4b5>
Function: function_to_evaluate at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def function_to_evaluate(stuff_to_evaluate):
     5         1         52.0     52.0    100.0      pass

